In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import shutil
from time import time
import torch
from copy import copy
from glob import glob
from PIL import Image, ImageFile
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import recgn_utils # utility script 

In [ ]:
# Verifica se CUDA está disponível 
gpu_on = torch.cuda.is_available()

if not gpu_on:
    print('Use a CPU. CUDA não está disponível...')
else:
    print('Use a GPU. CUDA está disponível...')

In [ ]:
# Configure alguns parametros:

# Path raiz do dataset 
rcgn_dir = '../input/dog-breed-recognition-v3/dogs/recognition'

# Numero de classes a serem selecionadas para enroll (de 1 a 20)
num_classes_enroll = 5

# Numero medio de imagens selecionadas para enroll, em cada classe 
num_img_enroll = 11

# Total de imagens utilizadas no enroll
total_img_enroll = num_classes_enroll * num_img_enroll

# Tamanho do dataset
batch_size = num_classes_enroll #TBC

# Numero de workers
num_workers = 0

# Numero de epocas
num_epochs = 25

# Metadados e Modelo treinado com fine-tune, na Parte-1 
checkpoint_path = "../input/modelp1v9ep15nllloss/model_epoch_15_acc_84.4318_loss_0.5199.pth"

In [ ]:
# Paths contendo imagens originais para treino e teste
data_dir = rcgn_dir + '/enroll'
test_dir = rcgn_dir + '/test'

# Plota quantidade ordenada de imagens por classe, para verficar se estao desbalanceadas
_, _ = recgn_utils.check_class(data_dir)

In [ ]:
# Lista nomes de todas as classes
full_class_names = [item.split('/')[-2] for item in sorted(glob(data_dir + "/*/"))]
print(f'> Lista das classes disponiveis para enroll = {full_class_names}\n')

# Seleciona subset de classes para enroll, e.g. classes de 1 a 5
partial_class_names = full_class_names[0:num_classes_enroll]  
print(f'> Lista das classes selecionadas para enroll = {partial_class_names}')

In [ ]:
# Seleciona as classes para treinamento/validacao no enroll
# Copia arquivos apenas das classes selecionadas
def sel_class(class_names, from_path, to_path):
    print(f'Copiando {len(class_names)} classes de {from_path}/enroll/ para {to_path}/enroll/')
    print(f'Copiando {len(class_names)} classes de {from_path}/test/ para {to_path}/test/')
    for name in class_names:
        old_path_train = (from_path + '/enroll/' + name)
        old_path_test = (from_path + '/test/' + name)
        new_path_train = (to_path + '/enroll/' + name)
        new_path_test = (to_path + '/test/' + name)
        shutil.copytree(old_path_train, new_path_train)
        shutil.copytree(old_path_test, new_path_test)

In [ ]:
# Define diretorios para transferencia de imagens
new_rcgn_dir = '/kaggle/working/rcgn_sample'

# Esvazia diretorio de destino (i.e. apaga e recria)
!rm -rf {new_rcgn_dir}
!mkdir {new_rcgn_dir}

# Realiza transferencia das classes selecionadas, para novo diretorio de desino
from_path = rcgn_dir
to_path = new_rcgn_dir
sel_class(partial_class_names, from_path, to_path)

# Lista novo diretorio de imagens
!ls {new_rcgn_dir}

In [ ]:
# NOVOS paths contendo imagens para treino (enroll) e teste
data_dir = new_rcgn_dir + '/enroll'
test_dir = new_rcgn_dir + '/test'

# Lista nomes das classes selecionadas
partial_class_names = [item.split('/')[-2] for item in sorted(glob(data_dir + "/*/"))]
print(f'> Lista das classes selecionadas para enroll = {partial_class_names}\n')

# Calcula numero de classes
num_classes_enroll = len(partial_class_names)
print(f'> Numero de classes selecionadas = {num_classes_enroll}')

In [ ]:
# Cria datasets de imagens  de treino e teste   
ds_train = datasets.ImageFolder(data_dir)
ds_test = datasets.ImageFolder(test_dir)

# Calcula total de imagens
total_img = len(ds_train)
print(f'> Numero de total de imagens disponiveis para enroll = {total_img}')

In [ ]:
# Define tamanho dos datasets de acordo com parametros iniciais 
# Isto eh: total_img_enroll = num_classes_enroll * num_img_enroll

train_size = math.floor(num_img_enroll * 0.9) * num_classes_enroll
valid_size = math.ceil(num_img_enroll * 0.1) * num_classes_enroll
rest_size = total_img - train_size - valid_size

train_set, val_set, _ = random_split(ds_train, [train_size, valid_size, rest_size],
                                            torch.Generator().manual_seed(2147483647))

print(f'Numero final de imagens de treinamento: {len(train_set)}')
print(f'Numero final de imagens de validacao: {len(val_set)}')
print(f'Numero final de imagens para enroll = {train_size + valid_size}')

In [ ]:
# Define valores mean e std para normalizar as imagens
# TODO: Valores baseados no ImageNet. Idealmente calcular o mean e std do dataset original
img_mean = np.array((0.485, 0.456, 0.406))
img_std = np.array((0.229, 0.224, 0.225))

# Define e aplica transformações nos datasets de treinamento, validação e teste
train_set.dataset = copy(ds_train)
train_set.dataset.transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomRotation(10),
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(img_mean, 
                                                         img_std)])
val_set.dataset.transform = transforms.Compose([transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(img_mean, 
                                                         img_std)])

test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(img_mean, 
                                                         img_std)])

test_set = datasets.ImageFolder(test_dir, transform=test_transforms)
print(f'Numero final de imagens de testes:', len(test_set))

# Cria conjunto de loaders
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

In [ ]:
# Exibe algumas imagens do loader com correspondentes labels

meanm = np.mean(img_mean)
stdm = np.mean(img_std)        
recgn_utils.sample_img_show(train_loader, partial_class_names, meanm, stdm)

In [ ]:
# Funcao que carrega modelo treinado com fine-tune, na Parte-1 

def load_model(checkpoint_path):
    chpt_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    chpt_out_features = 100
    # Recria model class
    model = models.resnet152(pretrained=True)
    classifier = nn.Sequential(nn.Linear(model.fc.in_features, 512),
                                  nn.ReLU(),
                                  nn.Linear(512, 256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, chpt_out_features),
                                  nn.LogSoftmax(dim=1))
    model.fc = classifier
    model.load_state_dict(chpt_dict['model_state_dict'])
    return model

In [ ]:
# Carrega modelo treinado com fine-tune, na Parte-1 
model = load_model(checkpoint_path)
print(model.fc)

In [ ]:
# Cria novo classificador

# Congela camadas para treinamento (feature extraction)
for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(nn.Linear(2048, 512), # model.fc.in_features = 2048
                              nn.ReLU(),
                              nn.Linear(512, 256),
                              nn.ReLU(),
                              nn.Dropout(0.5),
                              nn.Linear(256, num_classes_enroll),
                              nn.LogSoftmax(dim=1))
model.fc = classifier

# Define loss function (categorical cross-entropy)
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
criterion = nn.NLLLoss()

# Define otimizador de treinamento e diferentes taxas de aprendizado ao longo da rede
# https://pytorch.org/docs/stable/generated/torch.optim.SGD.html
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

if gpu_on:
    model.cuda()

In [ ]:
# Treina modelo 
start = time()

# Considera 3 condicoes de parada:
# 1) Valid loss medio crescente
# 2) Numero de epocas = num_epochs
# 3) Tempo maximo = total_img_enroll - 1

max_time = total_img_enroll - 1
model = recgn_utils.train_model(model, criterion, optimizer, loaders, num_epochs, 
                                gpu_on, max_time)

end = time()
print(f'Tempo total (aprox.) = {end - start} segundos') 
print(f'Tempo medio por imagem (aprox.) = {(end-start)/total_img} segundos') 

In [ ]:
# Testa modelo treinado com loader de testes

prob_pass, prob_fail = recgn_utils.test_model(model, criterion, test_loader, gpu_on)

In [ ]:
# Plota distribuição de probabilidades nos cassos de pass e fail do teste   

plt.hist(prob_fail, bins = np.arange(0,1.05,0.05)) 
plt.hist(prob_pass, bins = np.arange(0,1.05,0.05), alpha = 0.7) 
labels= ["Fail","Pass"]
plt.legend(labels)
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.title('Max outputs')

In [ ]:
# Seleciona uma foto aleatoria e testa o modelo

enroll_dir = './rcgn_sample/test/*/*'
enroll_data = np.array(glob(enroll_dir))
img_path = np.random.choice(enroll_data, 1)[0]
recgn_utils.imshow(img_path)
print(f'Foto selecionada aleatoriamente em:{img_path}')
pred_breed, pred_prob = recgn_utils.predict_breed_dog(model, partial_class_names, img_mean, img_std, img_path, gpu_on)
print(f'Probabilidade de {pred_prob*100:.2f}% de ser um {pred_breed}')